In [8]:
import pandas as pd
import glob
import os
import schedule
import time
from datetime import datetime
import win32com.client as win32

In [9]:
#Extrair arquivo
def pegar_ultimo_arquivo():
    # Procura todos os arquivos Excel que começam com 'relatorio_sujo_'
    arquivos = glob.glob('relatorio_sujo_*.xlsx')
    
    if not arquivos:
        print("Nenhum arquivo encontrado!")
        return None
    
    # Ordena pelo tempo de modificação (pega o mais recente)
    ultimo_arquivo = max(arquivos, key=os.path.getmtime)
    print(f"Lendo o arquivo: {ultimo_arquivo}")
    return ultimo_arquivo


In [10]:
#Tramento
def tratar_dados(df):
    print("Iniciando tratamento...")
    
    linhas_antes = len(df)
    df = df.drop_duplicates()
    print(f"- Duplicatas removidas: {linhas_antes - len(df)} linhas.")

    
    # Preencher com "Desconhecido"
    df['Nome'] = df['Nome'].fillna('Desconhecido')

    # Corrigir Valores Negativos
    df['Valor Gasto'] = df['Valor Gasto'].abs()

    #Padronizar Cidades (Espaços e Maiúsculas)
    df['Cidade'] = df['Cidade'].str.strip().str.title()

    # Padronizar Datas
    df['Data de nascimento'] = pd.to_datetime(df['Data de nascimento'], errors='coerce')
    df['Data de nascimento'] = df['Data de nascimento'].dt.strftime('%d/%m/%Y')


    return df


In [11]:
def enviar_email_com_anexo(nome_arquivo, faturamento, qtd_vendas, ticket_medio):
    try:
        outlook = win32.Dispatch('outlook.application')
        email = outlook.CreateItem(0)
        email.To = "jean.souzaper@gmail.com"
        email.Subject = "Relatório Diário de Vendas e Produtividade"
        
        # Formatação do corpo do e-mail com os cálculos
        email.HTMLBody = f"""
        <html>
            <body>
                <p>Olá,</p>
                <p>Segue abaixo o resumo de produtividade:</p>
                
                <ul>
                    <li><strong>Quantidade de Vendas:</strong> {qtd_vendas}</li>
                    <li><strong>Faturamento Total:</strong> R$ {faturamento:,.2f}</li>
                    <li><strong>Ticket Médio:</strong> R$ {ticket_medio:,.2f}</li>
                </ul>

                <p>O relatório detalhado com os dados limpos segue em anexo: <strong>{nome_arquivo}</strong></p>
                
                <p>Att,<br>Bot de Automação</p>
            </body>
        </html>
        """
        
        # Anexa o arquivo
        caminho_completo = os.path.join(os.getcwd(), nome_arquivo)
        email.Attachments.Add(caminho_completo)
        
        email.Send()
        print(f"E-mail enviado com sucesso com os KPIs!")
        
    except Exception as e:
        print(f"Erro ao enviar email: {e}")

In [12]:
def executar_pipeline():
    arquivo = pegar_ultimo_arquivo()

    if arquivo:
        try:
            print(f"Processando: {arquivo}")
            
            # Lê o Excel
            df = pd.read_excel(arquivo)

            # Trata os dados
            df_tratado = tratar_dados(df)

            # --- CÁLCULOS DE PRODUTIVIDADE ---
            faturamento_total = df_tratado['Valor Gasto'].sum()
            quantidade_vendas = len(df_tratado)
            
            # Evita divisão por zero
            if quantidade_vendas > 0:
                ticket_medio = faturamento_total / quantidade_vendas
            else:
                ticket_medio = 0

            # Salva o arquivo NOVO com os dados LIMPOS
            nome_limpo = "CLEAN_" + os.path.basename(arquivo)
            df_tratado.to_excel(nome_limpo, index=False)
            print(f"Arquivo limpo salvo como: {nome_limpo}")

            # Marca o arquivo VELHO como processado
            os.rename(arquivo, arquivo + ".processado")
            print("Arquivo original marcado como processado.")
            
            # Retorna todos os dados necessários para o e-mail
            return nome_limpo, faturamento_total, quantidade_vendas, ticket_medio

        except Exception as e:
            print(f"Erro no pipeline: {e}")
            return None
    
    return None

In [13]:
def job_completo():
    # Roda o pipeline e captura os múltiplos valores retornados
    resultado = executar_pipeline()
    
    # Se resultado não for None (ou seja, deu certo)
    if resultado:
        nome_arquivo, faturamento, qtd, ticket = resultado
        enviar_email_com_anexo(nome_arquivo, faturamento, qtd, ticket)
    else:
        print("Nenhum arquivo novo para processar.")    
        pass

In [ ]:
schedule.every(10).seconds.do(job_completo)

while True:
    schedule.run_pending()
    time.sleep(1)

Lendo o arquivo: relatorio_sujo_15-12-2025_15-27-07.xlsx
Processando: relatorio_sujo_15-12-2025_15-27-07.xlsx
Iniciando tratamento...
- Duplicatas removidas: 10 linhas.
Arquivo limpo salvo como: CLEAN_relatorio_sujo_15-12-2025_15-27-07.xlsx
Arquivo original marcado como processado.


C:\Users\jeans\AppData\Local\Temp\ipykernel_22856\2687466554.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Nome'] = df['Nome'].fillna('Desconhecido')
C:\Users\jeans\AppData\Local\Temp\ipykernel_22856\2687466554.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Valor Gasto'] = df['Valor Gasto'].abs()
C:\Users\jeans\AppData\Local\Temp\ipykernel_22856\2687466554.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

E-mail enviado com sucesso com os KPIs!


KeyboardInterrupt: 